In [3]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Ruta raíz donde están las carpetas con archivos .tif
ruta_raiz = "C:/Users/aloac/Documents/reproyeccion/prueba"

# EPSG de destino
epsg_destino = 32614

def reproyectar_raster(archivo_tif):
    # Nombre temporal para evitar conflictos al sobrescribir
    archivo_temp = archivo_tif.replace(".tif", "_temp.tif")
    
    with rasterio.open(archivo_tif) as src:
        transform, width, height = calculate_default_transform(
            src.crs, f"EPSG:{epsg_destino}", src.width, src.height, *src.bounds
        )
        perfil = src.profile.copy()
        perfil.update({
            "crs": f"EPSG:{epsg_destino}",
            "transform": transform,
            "width": width,
            "height": height
        })

        with rasterio.open(archivo_temp, "w", **perfil) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=f"EPSG:{epsg_destino}",
                    resampling=Resampling.nearest
                )

    # Cerrar los archivos antes de reemplazar
    os.remove(archivo_tif)  # Eliminar el original
    os.rename(archivo_temp, archivo_tif)  # Renombrar el reproyectado

# Recorrer las carpetas y procesar los archivos .tif
for carpeta_raiz, subcarpetas, archivos in os.walk(ruta_raiz):
    for archivo in archivos:
        if archivo.endswith(".tif"):
            ruta_completa = os.path.join(carpeta_raiz, archivo)
            print(f"Reproyectando: {ruta_completa}")
            reproyectar_raster(ruta_completa)
            print(f"✔ {archivo} reproyectado y sobrescrito.")

Reproyectando: C:/Users/aloac/Documents/reproyeccion/prueba\D1-02-05_Susceptibilidad a Hundimientos\R15065_hundimientos_USGS_2024.tif
✔ R15065_hundimientos_USGS_2024.tif reproyectado y sobrescrito.
Reproyectando: C:/Users/aloac/Documents/reproyeccion/prueba\D1-02-05_Susceptibilidad a Inestabilidad de Laderas\R15065_inestabilidad_ladera_USGS_2024.tif
✔ R15065_inestabilidad_ladera_USGS_2024.tif reproyectado y sobrescrito.
Reproyectando: C:/Users/aloac/Documents/reproyeccion/prueba\D1-02-07_Precipitación media diaria para un periodo de retorno de 10, 50 y 100 años\R15065_precipitacion_media_diaria_100a_UNAM_2014.tif
✔ R15065_precipitacion_media_diaria_100a_UNAM_2014.tif reproyectado y sobrescrito.
Reproyectando: C:/Users/aloac/Documents/reproyeccion/prueba\D1-02-07_Precipitación media diaria para un periodo de retorno de 10, 50 y 100 años\R15065_precipitacion_media_diaria_10a_UNAM_2014.tif
✔ R15065_precipitacion_media_diaria_10a_UNAM_2014.tif reproyectado y sobrescrito.
Reproyectando: C:/